<a href="https://colab.research.google.com/github/Aravindh4404/FYPSeagullClassification01/blob/main/REFINEMENTVGG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image
from torchvision import transforms
from pathlib import Path
from tqdm import tqdm
from datetime import datetime
import seaborn as sns

# -------------------------------------------------------------------------
# 1) Define the VGG16Modified Class (as per your training)
# -------------------------------------------------------------------------
class VGG16Modified(nn.Module):
    def __init__(self):
        super(VGG16Modified, self).__init__()
        from torchvision.models import VGG16_Weights
        self.vgg = models.vgg16(weights=VGG16_Weights.IMAGENET1K_V1)
        num_ftrs = self.vgg.classifier[6].in_features
        self.vgg.classifier[6] = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(num_ftrs, 2)  # Binary classification (2 classes)
        )

    def forward(self, x):
        return self.vgg(x)

# -------------------------------------------------------------------------
# 2) Load Model Checkpoint
# -------------------------------------------------------------------------
checkpoint_path = "/content/drive/My Drive/FYP/VGGModel/HQ3latst_20250216/checkpoint_model_vgg_20250216.pth"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = VGG16Modified().to(device)
model.eval()

if not os.path.exists(checkpoint_path):
    print(f"ERROR: The checkpoint file was not found at:\n  {checkpoint_path}\nPlease check the path or filename and try again.")
    exit()
else:
    model.load_state_dict(torch.load(checkpoint_path, map_location=device))
    print(f"Successfully loaded checkpoint from {checkpoint_path}")

# -------------------------------------------------------------------------
# 3) Define Class Names and Transformations
# -------------------------------------------------------------------------
# IMPORTANT: Ensure that the folder names in your dataset match these class names.
class_names = ['Glaucous_Winged_Gull', 'Slaty_Backed_Gull']
# (If needed, update the names to exactly match your folder names.)

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # VGG expects 224x224 input
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# -------------------------------------------------------------------------
# 4) Grad-CAM Implementation
# -------------------------------------------------------------------------
def generate_gradcam(model, image_tensor, target_layer):
    """
    Generates Grad-CAM for a given image and model.
    Returns:
      cam (normalized 2D numpy array),
      predicted_class_idx (int),
      predicted_confidence (float)  # new
    """
    model.eval()
    features = []
    grads = []

    def forward_hook(module, input, output):
        features.append(output)

    def backward_hook(module, grad_in, grad_out):
        grads.append(grad_out[0])

    # Register hooks on the target convolutional layer
    forward_handle = target_layer.register_forward_hook(forward_hook)
    backward_handle = target_layer.register_backward_hook(backward_hook)

    # Forward pass
    outputs = model(image_tensor)
    # Compute softmax to get confidence
    probs = F.softmax(outputs, dim=1)
    predicted_class_idx = torch.argmax(probs, dim=1).item()
    predicted_confidence = probs[0, predicted_class_idx].item()  # get confidence for predicted class

    # Backward pass for the predicted class
    model.zero_grad()
    class_score = outputs[0, predicted_class_idx]
    class_score.backward()

    # Extract the gradients and the feature maps
    gradient = grads[0].detach().cpu().numpy()[0]  # shape: (C, H, W)
    feature_map = features[0].detach().cpu().numpy()[0]  # shape: (C, H, W)

    # Remove hooks
    forward_handle.remove()
    backward_handle.remove()

    # Compute the weights: global-average pooling of the gradients
    weights = np.mean(gradient, axis=(1, 2))  # shape: (C,)

    # Generate CAM as a weighted combination of feature maps
    cam = np.zeros(feature_map.shape[1:], dtype=np.float32)
    for i, w in enumerate(weights):
        cam += w * feature_map[i, :, :]

    # Apply ReLU and normalize the CAM
    cam = np.maximum(cam, 0)
    cam = cv2.resize(cam, (image_tensor.shape[3], image_tensor.shape[2]))
    cam -= np.min(cam)
    cam /= (cam.max() + 1e-9)

    return cam, predicted_class_idx, predicted_confidence

# -------------------------------------------------------------------------
# 5) Utility Functions
# -------------------------------------------------------------------------
def preprocess_image(image_path):
    """
    Loads an image and applies transformations.
    Returns a tensor of shape (1, 3, 224, 224) on the specified device.
    """
    image = Image.open(image_path).convert("RGB")
    return transform(image).unsqueeze(0).to(device)

def save_visualization(image_path, cam, predicted_class, confidence, true_class, save_path):
    """
    Creates an overlay of the Grad-CAM heatmap on the original image,
    adds text annotations (true class, predicted class, correctness, confidence), and
    saves the result to 'save_path'.
    """
    # Load and resize original image
    original_img = Image.open(image_path).resize((224, 224), Image.LANCZOS)
    original_np = np.array(original_img)

    # Create heatmap from CAM
    heatmap = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)
    heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)

    # Blend heatmap with the original image
    overlay = np.clip(0.5 * original_np + 0.5 * heatmap, 0, 255).astype(np.uint8)

    # Prepare text to overlay
    pred_label = class_names[predicted_class]
    correct = (predicted_class == class_names.index(true_class))
    confidence_text = f"{confidence*100:.2f}%"
    result_text = f"True: {true_class} | Pred: {pred_label} | Conf: {confidence_text} | {'Correct' if correct else 'Wrong'}"

    # Add text overlay using cv2.putText; convert image to BGR for OpenCV
    overlay_bgr = cv2.cvtColor(overlay, cv2.COLOR_RGB2BGR)
    cv2.putText(
        overlay_bgr,
        result_text,
        (10, 25),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.6,
        (0, 255, 0) if correct else (0, 0, 255),
        2,
        cv2.LINE_AA
    )
    # Convert back to RGB for saving
    overlay_rgb = cv2.cvtColor(overlay_bgr, cv2.COLOR_BGR2RGB)

    # Save the image
    cv2.imwrite(save_path, cv2.cvtColor(overlay_rgb, cv2.COLOR_RGB2BGR))

def save_statistics_csv(stat_list, csv_path):
    """
    Saves the list of statistics (one dict per image) as a CSV file.
    """
    df = pd.DataFrame(stat_list)
    df.to_csv(csv_path, index=False)
    print(f"Statistics saved to {csv_path}")

# -------------------------------------------------------------------------
# 6) Process Dataset Folder with Two Classes and Save Results
# -------------------------------------------------------------------------
def process_dataset(dataset_path, output_base, delete_wrong=True):
    """
    Processes images stored in subfolders (each subfolder name is taken as the true class)
    and generates Grad-CAM visualizations with text overlays.

    The output is saved to Google Drive under 'output_base/TIMESTAMP/'.
    The folder structure created is:
       TIMESTAMP/
           <true_class>/
               correct/   (images that were correctly classified)
               wrong/     (images that were misclassified)

    Also collects statistics (true class, predicted class, file name, confidence, etc.) for later review.

    Parameters:
      dataset_path (str/Path): The directory containing two subfolders (each named after a class).
      output_base (str/Path) : Base folder to store the output results.
      delete_wrong (bool)    : If True, the code will delete the original file when it is wrongly predicted.
    """
    dataset_path = Path(dataset_path)
    if not dataset_path.exists():
        print(f"ERROR: Dataset path not found: {dataset_path}")
        return

    # Create an output directory with a timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_dir = Path(output_base) / timestamp
    output_dir.mkdir(parents=True, exist_ok=True)
    print(f"Results will be saved to: {output_dir}")

    # Prepare statistics list
    stats_list = []

    # Process each subfolder (each is a true class)
    for subfolder in sorted(dataset_path.iterdir()):
        if not subfolder.is_dir():
            continue
        true_class = subfolder.name
        if true_class not in class_names:
            print(f"Skipping folder '{true_class}' as it is not in class_names")
            continue

        # Create output subfolders for this true class (correct & wrong)
        out_class_dir = output_dir / true_class
        correct_dir = out_class_dir / "correct"
        wrong_dir = out_class_dir / "wrong"
        correct_dir.mkdir(parents=True, exist_ok=True)
        wrong_dir.mkdir(parents=True, exist_ok=True)

        # Process each image file in this class folder
        # (support jpg, jpeg, png; you can extend below)
        image_files = list(subfolder.glob("*.[jJ][pP][gG]")) + \
                      list(subfolder.glob("*.[jJ][pP][eE][gG]")) + \
                      list(subfolder.glob("*.[pP][nN][gG]"))
        print(f"Processing {len(image_files)} images in folder '{true_class}'...")

        # Use the last convolutional layer of VGG for Grad-CAM
        target_layer = model.vgg.features[-1]

        for image_path in tqdm(image_files, desc=f"Processing {true_class}"):
            try:
                # Preprocess image
                image_tensor = preprocess_image(image_path)

                # Generate Grad-CAM and get prediction + confidence
                cam, predicted_class_idx, predicted_confidence = generate_gradcam(model, image_tensor, target_layer)
                predicted_label = class_names[predicted_class_idx]
                is_correct = (predicted_label == true_class)

                # Determine save directory based on correctness
                save_subdir = correct_dir if is_correct else wrong_dir
                save_filename = f"{image_path.stem}_gradcam.png"
                save_path = str(save_subdir / save_filename)

                # Save the visualization (with overlay text)
                save_visualization(str(image_path), cam, predicted_class_idx, predicted_confidence, true_class, save_path)

                # Append to statistics
                stats_list.append({
                    "file": str(image_path),
                    "true_class": true_class,
                    "predicted_class": predicted_label,
                    "confidence": predicted_confidence,
                    "correct": is_correct,
                    "output_file": save_path
                })

                # If wrong, delete the original image from the dataset
                if (not is_correct) and delete_wrong:
                    # Caution: This permanently removes the file from disk
                    os.remove(str(image_path))
                    # If you prefer to move it instead of deleting, use:
                    # import shutil
                    # shutil.move(str(image_path), str(wrong_dir / image_path.name))

            except Exception as e:
                print(f"Error processing {image_path}: {e}")

    # Save statistics as CSV in the output directory
    csv_path = output_dir / "statistics.csv"
    save_statistics_csv(stats_list, csv_path)

    # Optionally, generate and save a confusion matrix
    try:
        df = pd.DataFrame(stats_list)
        if not df.empty:
            confusion = pd.crosstab(df['true_class'], df['predicted_class'], rownames=['True'], colnames=['Predicted'])
            plt.figure(figsize=(6, 4))
            sns.heatmap(confusion, annot=True, fmt="d", cmap="Blues")
            plt.title("Confusion Matrix")
            cm_path = output_dir / "confusion_matrix.png"
            plt.savefig(cm_path, dpi=300, bbox_inches='tight')
            plt.close()
            print(f"Confusion matrix saved to: {cm_path}")
    except Exception as e:
        print(f"Error generating confusion matrix: {e}")

# -------------------------------------------------------------------------
# 7) Main Execution
# -------------------------------------------------------------------------
if __name__ == "__main__":
    # Specify the parent folder containing your two class subfolders.
    dataset_path = "/content/drive/My Drive/FYP/Black Background/"

    # Specify the base output directory (on Google Drive or local).
    output_base = "/content/drive/My Drive/FYP/VGGAnalysis"

    # Set delete_wrong=True if you want to permanently delete wrongly predicted images.
    process_dataset(dataset_path, output_base, delete_wrong=True)


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:03<00:00, 142MB/s]
<ipython-input-1-87564d99d425>:47: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. P

Successfully loaded checkpoint from /content/drive/My Drive/FYP/VGGModel/HQ3latst_20250216/checkpoint_model_vgg_20250216.pth
Results will be saved to: /content/drive/My Drive/FYP/VGGAnalysis/20250218_040548
Processing 124 images in folder 'Glaucous_Winged_Gull'...


Processing Glaucous_Winged_Gull: 100%|██████████| 124/124 [00:26<00:00,  4.62it/s]


Processing 143 images in folder 'Slaty_Backed_Gull'...


Processing Slaty_Backed_Gull: 100%|██████████| 143/143 [01:41<00:00,  1.40it/s]


Statistics saved to /content/drive/My Drive/FYP/VGGAnalysis/20250218_040548/statistics.csv
Confusion matrix saved to: /content/drive/My Drive/FYP/VGGAnalysis/20250218_040548/confusion_matrix.png


In [ ]:
import os
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image
from torchvision import transforms
from pathlib import Path
from tqdm import tqdm
from datetime import datetime
import seaborn as sns

# -------------------------------------------------------------------------
# 1) Define the VGG16Modified Class (as per your training)
# -------------------------------------------------------------------------
class VGG16Modified(nn.Module):
    def __init__(self):
        super(VGG16Modified, self).__init__()
        from torchvision.models import VGG16_Weights
        self.vgg = models.vgg16(weights=VGG16_Weights.IMAGENET1K_V1)
        num_ftrs = self.vgg.classifier[6].in_features
        self.vgg.classifier[6] = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(num_ftrs, 2)  # Binary classification
        )

    def forward(self, x):
        return self.vgg(x)

# -------------------------------------------------------------------------
# 2) Load Model Checkpoint
# -------------------------------------------------------------------------
checkpoint_path = "/content/drive/My Drive/FYP/VGGModel/HQ3latst_20250216/checkpoint_model_vgg_20250216.pth"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = VGG16Modified().to(device)
model.eval()

if not os.path.exists(checkpoint_path):
    print(f"ERROR: The checkpoint file was not found at:\n  {checkpoint_path}\nPlease check the path or filename and try again.")
    exit()
else:
    model.load_state_dict(torch.load(checkpoint_path, map_location=device))
    print(f"Successfully loaded checkpoint from {checkpoint_path}")

# -------------------------------------------------------------------------
# 3) Define Class Names and Transformations
# -------------------------------------------------------------------------
# IMPORTANT: Ensure that the folder names in your dataset match these class names.
class_names = ['Glaucous_Winged_Gull', 'Slaty_Backed_Gull']
# (If needed, update the names to exactly match your folder names.)

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # VGG expects 224x224 input
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# -------------------------------------------------------------------------
# 4) Grad-CAM Implementation
# -------------------------------------------------------------------------
def generate_gradcam(model, image_tensor, target_layer):
    """
    Generates Grad-CAM for a given image and model.
    Returns:
      cam (normalized 2D numpy array),
      predicted_class_idx (int)
    """
    model.eval()
    features = []
    grads = []

    def forward_hook(module, input, output):
        features.append(output)

    def backward_hook(module, grad_in, grad_out):
        grads.append(grad_out[0])

    # Register hooks on the target convolutional layer
    forward_handle = target_layer.register_forward_hook(forward_hook)
    backward_handle = target_layer.register_backward_hook(backward_hook)

    # Forward pass
    outputs = model(image_tensor)
    predicted_class_idx = outputs.argmax(dim=1).item()

    # Backward pass for the predicted class
    model.zero_grad()
    class_score = outputs[0, predicted_class_idx]
    class_score.backward()

    # Extract the gradients and the feature maps
    gradient = grads[0].detach().cpu().numpy()[0]  # shape: (C, H, W)
    feature_map = features[0].detach().cpu().numpy()[0]  # shape: (C, H, W)

    # Remove hooks
    forward_handle.remove()
    backward_handle.remove()

    # Compute the weights: global-average pooling of the gradients
    weights = np.mean(gradient, axis=(1, 2))  # shape: (C,)

    # Generate CAM as a weighted combination of feature maps
    cam = np.zeros(feature_map.shape[1:], dtype=np.float32)
    for i, w in enumerate(weights):
        cam += w * feature_map[i, :, :]

    # Apply ReLU and normalize the CAM
    cam = np.maximum(cam, 0)
    cam = cv2.resize(cam, (image_tensor.shape[3], image_tensor.shape[2]))
    cam -= np.min(cam)
    cam /= (cam.max() + 1e-9)

    return cam, predicted_class_idx

# -------------------------------------------------------------------------
# 5) Utility Functions
# -------------------------------------------------------------------------
def preprocess_image(image_path):
    """
    Loads an image and applies transformations.
    Returns a tensor of shape (1, 3, 224, 224) on the specified device.
    """
    image = Image.open(image_path).convert("RGB")
    return transform(image).unsqueeze(0).to(device)

def save_visualization(image_path, cam, predicted_class, true_class, save_path):
    """
    Creates an overlay of the Grad-CAM heatmap on the original image,
    adds text annotations (true class, predicted class, correctness), and
    saves the result to 'save_path'.
    """
    # Load and resize original image
    original_img = Image.open(image_path).resize((224, 224), Image.LANCZOS)
    original_np = np.array(original_img)

    # Create heatmap from CAM
    heatmap = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)
    heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)

    # Blend heatmap with the original image
    overlay = np.clip(0.5 * original_np + 0.5 * heatmap, 0, 255).astype(np.uint8)

    # Prepare text to overlay
    pred_label = class_names[predicted_class]
    correct = (predicted_class == class_names.index(true_class))
    result_text = f"True: {true_class} | Pred: {pred_label} | {'Correct' if correct else 'Wrong'}"

    # Add text overlay using cv2.putText; convert image to BGR for OpenCV
    overlay_bgr = cv2.cvtColor(overlay, cv2.COLOR_RGB2BGR)
    cv2.putText(overlay_bgr, result_text, (10, 25), cv2.FONT_HERSHEY_SIMPLEX,
                0.7, (0, 255, 0) if correct else (0, 0, 255), 2, cv2.LINE_AA)
    # Convert back to RGB for saving
    overlay_rgb = cv2.cvtColor(overlay_bgr, cv2.COLOR_BGR2RGB)

    # Save the image
    cv2.imwrite(save_path, cv2.cvtColor(overlay_rgb, cv2.COLOR_RGB2BGR))

def save_statistics_csv(stat_list, csv_path):
    """
    Saves the list of statistics (one dict per image) as a CSV file.
    """
    df = pd.DataFrame(stat_list)
    df.to_csv(csv_path, index=False)
    print(f"Statistics saved to {csv_path}")

# -------------------------------------------------------------------------
# 6) Process Dataset Folder with Two Classes and Save Results
# -------------------------------------------------------------------------
def process_dataset(dataset_path, output_base):
    """
    Processes images stored in subfolders (each subfolder name is taken as the true class)
    and generates Grad-CAM visualizations with text overlays.

    The output is saved to Google Drive under 'output_base/TIMESTAMP/'.
    The folder structure created is:
       TIMESTAMP/
           <true_class>/
               correct/   (images that were correctly classified)
               wrong/     (images that were misclassified)

    Also collects statistics (true class, predicted class, file name, etc.) for later review.
    """
    dataset_path = Path(dataset_path)
    if not dataset_path.exists():
        print(f"ERROR: Dataset path not found: {dataset_path}")
        return

    # Create an output directory with a timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_dir = Path(output_base) / timestamp
    output_dir.mkdir(parents=True, exist_ok=True)
    print(f"Results will be saved to: {output_dir}")

    # Prepare statistics list
    stats_list = []

    # Process each subfolder (each is a true class)
    for subfolder in sorted(dataset_path.iterdir()):
        if not subfolder.is_dir():
            continue
        true_class = subfolder.name
        if true_class not in class_names:
            print(f"Skipping folder '{true_class}' as it is not in class_names")
            continue

        # Create output subfolders for this true class (correct & wrong)
        out_class_dir = output_dir / true_class
        correct_dir = out_class_dir / "correct"
        wrong_dir = out_class_dir / "wrong"
        correct_dir.mkdir(parents=True, exist_ok=True)
        wrong_dir.mkdir(parents=True, exist_ok=True)

        # Process each image file in this class folder
        image_files = list(subfolder.glob("*.[jJ][pP][gG]")) + list(subfolder.glob("*.[pP][nN][gG]"))
        print(f"Processing {len(image_files)} images in folder '{true_class}'...")

        # Use the last convolutional layer of VGG for Grad-CAM
        target_layer = model.vgg.features[-1]

        for image_path in tqdm(image_files, desc=f"Processing {true_class}"):
            try:
                # Preprocess image
                image_tensor = preprocess_image(image_path)

                # Generate Grad-CAM and get prediction
                cam, predicted_class_idx = generate_gradcam(model, image_tensor, target_layer)
                predicted_label = class_names[predicted_class_idx]
                is_correct = (predicted_label == true_class)

                # Determine save directory based on correctness
                save_subdir = correct_dir if is_correct else wrong_dir
                save_filename = f"{image_path.stem}_gradcam.png"
                save_path = str(save_subdir / save_filename)

                # Save the visualization (with overlay text)
                save_visualization(str(image_path), cam, predicted_class_idx, true_class, save_path)

                # Append to statistics
                stats_list.append({
                    "file": str(image_path),
                    "true_class": true_class,
                    "predicted_class": predicted_label,
                    "correct": is_correct,
                    "output_file": save_path
                })
            except Exception as e:
                print(f"Error processing {image_path}: {e}")

    # Save statistics as CSV in the output directory
    csv_path = output_dir / "statistics.csv"
    save_statistics_csv(stats_list, csv_path)

    # Optionally, generate and save a confusion matrix
    try:
        df = pd.DataFrame(stats_list)
        if not df.empty:
            confusion = pd.crosstab(df['true_class'], df['predicted_class'], rownames=['True'], colnames=['Predicted'])
            plt.figure(figsize=(6, 4))
            sns.heatmap(confusion, annot=True, fmt="d", cmap="Blues")
            plt.title("Confusion Matrix")
            cm_path = output_dir / "confusion_matrix.png"
            plt.savefig(cm_path, dpi=300, bbox_inches='tight')
            plt.close()
            print(f"Confusion matrix saved to: {cm_path}")
    except Exception as e:
        print(f"Error generating confusion matrix: {e}")

# -------------------------------------------------------------------------
# 7) Main Execution
# -------------------------------------------------------------------------
if __name__ == "__main__":
    # Specify the parent folder containing your two class subfolders.
    dataset_path = "/content/drive/My Drive/FYP/Entire Bird Segment/"
    # Specify the base output directory (on Google Drive)
    output_base = "/content/drive/My Drive/FYP/VGGAnalysis"
    process_dataset(dataset_path, output_base)


<ipython-input-3-c430202279e6>:47: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(checkpoint_path, map_location=device))


Successfully loaded checkpoint from /content/drive/My Drive/FYP/VGGModel/HQ3latst_20250216/checkpoint_model_vgg_20250216.pth
Results will be saved to: /content/drive/My Drive/FYP/VGGAnalysis/20250216_153814
Processing 116 images in folder 'Glaucous_Winged_Gull'...


Processing Glaucous_Winged_Gull: 100%|██████████| 116/116 [04:42<00:00,  2.44s/it]


Processing 97 images in folder 'Slaty_Backed_Gull'...


Processing Slaty_Backed_Gull: 100%|██████████| 97/97 [05:33<00:00,  3.44s/it]


Statistics saved to /content/drive/My Drive/FYP/VGGAnalysis/20250216_153814/statistics.csv
Confusion matrix saved to: /content/drive/My Drive/FYP/VGGAnalysis/20250216_153814/confusion_matrix.png


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
